### Integrantes: Josué Deley, Martin Soto

# Banco Omega Dataset

El banco Omega ha observado un aumento en el número de clientes que entran en mora en sus pagos de tarjeta de crédito, lo que ha llevado a una disminución de sus ganancias. La entidad financiera desea entender las razones detrás de la mora de sus clientes y detectar los factores que influyen para determinar si una deuda será pagada o no. Para lograr este objetivo, el banco decide utilizar técnicas de análisis de datos y aprendizaje automático para procesar una gran cantidad de datos de clientes, incluyendo información personal, historial crediticio, patrones de gasto y pagos, entre otros. A partir de este análisis, el banco busca identificar los factores que podrían estar influyendo en la mora de sus clientes y construir un modelo de clasificación que permita determinar si un cliente en particular pagará su deuda a tiempo o no.

### Características del data set   
• ID: ID del cliente • CU: Cupo máximo de la tarjeta de crédito • G: Género del cliente. (M = Masculino, F = Femenino, O = Otros). • ED: Nivel de instrucción educativa (1 = primaria, 2 = universidad, 3 = secundaria, 4 = otros). • EC: Estado civil del cliente. (Soltero, Casado, Otros) • E: Edad del cliente. • M1 – M6: Historial de pago previo. M1 = historial de pago (marzo 2024), M2 = historial de pago (febrero 2024), …, M6 = historial de pago (octubre 2023). El historial de pago se define: -1 = paga debidamente, 1 = retraso en el pago por un mes, 2 = retraso en el pago por dos meses, ..., 8 = retraso en el pago por ocho meses, 9 = retraso en el pago por nueve meses o más. • D1 – D6: Monto del estado de cuenta. D1 = monto del estado de cuenta (marzo 2023), D2 = monto del estado de cuenta (febrero 2024), …, D6 = monto del estado de cuenta (octubre 2023). El monto del estado de cuenta se encuentra descrito en dólares americanos. • P1 – P6: Monto del pago previo. P1 = monto pagado (marzo 2024), P2 = monto pagado (febrero 2024), …, P6 = monto pagado (octubre 2023). • SP: Próximo pago en mora. (Si, No)

## 1.EDA

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

url = "https://raw.githubusercontent.com/chrisrueda/EndomorfismoII_2024/main/PROYECTOS/SEGUNDO%20PARCIAL/mora_cdto.csv"

data = pd.read_csv(url)

data.head(10)

,ID,CU,G,ED,EC,E,M1,M2,M3,M4,...,D4,D5,D6,P1,P2,P3,P4,P5,P6,SP
0,1,2000,F,2,Casado,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,68.9,0.0,0.0,0.0,0.0,Si
1,2,12000,F,2,Soltero,26,-1,2,0,0,...,327.2,345.5,326.1,0.0,100.0,100.0,100.0,0.0,200.0,Si
2,3,9000,F,2,Soltero,34,0,0,0,0,...,1433.1,1494.8,1554.9,151.8,150.0,100.0,100.0,100.0,500.0,No
3,4,5000,F,2,Casado,37,0,0,0,0,...,2831.4,2895.9,2954.7,200.0,201.9,120.0,110.0,106.9,100.0,No
4,5,5000,M,2,Casado,57,-1,0,-1,0,...,2094.0,1914.6,1913.1,200.0,3668.1,1000.0,900.0,68.9,67.9,No
5,6,5000,M,1,Soltero,37,0,0,0,0,...,1939.4,1961.9,2002.4,250.0,181.5,65.7,100.0,100.0,80.0,No
6,7,50000,M,1,Soltero,29,0,0,0,0,...,54265.3,48300.3,47394.4,5500.0,4000.0,3800.0,2023.9,1375.0,1377.0,No
7,8,10000,F,2,Soltero,23,0,-1,-1,0,...,22.1,-15.9,56.7,38.0,60.1,0.0,58.1,168.7,154.2,No
8,9,14000,F,3,Casado,28,0,0,2,0,...,1221.1,1179.3,371.9,332.9,0.0,43.2,100.0,100.0,100.0,No
9,10,2000,M,3,Soltero,35,-2,-2,-2,-2,...,0.0,1300.7,1391.2,0.0,0.0,0.0,1300.7,112.2,0.0,No


## 2. Análisis estadístico simple

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      30000 non-null  int64  
 1   CU      30000 non-null  int64  
 2   G       29997 non-null  object 
 3   ED      30000 non-null  int64  
 4   EC      29999 non-null  object 
 5   E       30000 non-null  int64  
 6   M1      30000 non-null  int64  
 7   M2      30000 non-null  int64  
 8   M3      30000 non-null  int64  
 9   M4      30000 non-null  int64  
 10  M5      30000 non-null  int64  
 11  M6      30000 non-null  int64  
 12  D1      30000 non-null  float64
 13  D2      30000 non-null  float64
 14  D3      30000 non-null  float64
 15  D4      30000 non-null  float64
 16  D5      30000 non-null  float64
 17  D6      30000 non-null  float64
 18  P1      30000 non-null  float64
 19  P2      30000 non-null  float64
 20  P3      30000 non-null  float64
 21  P4      30000 non-null  float64
 22

In [16]:
# Verificar rango de datos
for x in data.columns:
    print(x)
    print(data[x].unique(), "\n")

ID
[    1     2     3 ... 29998 29999 30000] 

CU
[  2000  12000   9000   5000  50000  10000  14000  20000  26000  63000
   7000  25000  32000  36000  18000  13000  45000   6000  23000  16000
  28000   1000   4000  21000  15000  38000  31000  40000   8000  29000
  34000  30000   3000  24000  47000  48000  35000  33000  11000  42000
  17000  37000  27000  22000  19000  51000  46000  44000  41000  49000
  39000  58000  60000  62000  61000  70000  67000  68000  43000  55000
  54000 100000  53000  71000  56000  52000  75000  64000   1600  57000
  59000  66000  72000  32768  74000  80000  76000  69000  65000  78000
  73000] 

G
['F' 'M' 'O' nan] 

ED
[2 1 3 5 4 6 0] 

EC
['Casado' 'Soltero' 'Otros' nan '0'] 

E
[24 26 34 37 57 29 23 28 35 51 41 30 49 39 40 27 47 33 32 54 58 22 25 31
 46 42 43 45 56 44 53 38 63 36 52 48 55 60 50 75 61 73 59 21 67 66 62 70
 72 64 65 71 69 68 79 74] 

M1
[ 2 -1  0 -2  1  3  4  8  7  5  6] 

M2
[ 2  0 -1 -2  3  5  7  4  1  6  8] 

M3
[-1  0  2 -2  3  4  6  7  1

#### Características a modificar: ['G', 'ED', 'EC', 'M1-M6', 'D1-D6', 'P1-P6', 'SP']

G: Posee datos NaN y cambiar tipo de dato.

ED: Posee datos fuera del rango [1, 2, 3, 4]    Corregir [0, 5, 6]

EC: Posee datos NaN, y datos fuera del rango ['Casado' 'Soltero' 'Otros']   Corregir ['nan' '0'] y cambiar tipo de dato.

M1-M6: Datos fuera del rango [-1, 1, 2, ..., 9]     Corregir [-2, 0]

D1-D6: corregir tipo de datos y verificar números negativos.

P1-P6: Corregir tipo de datos y verificar números negativos.

SP: cambiar tipo de dato.


### Analizar el balance de los datos

In [21]:
# Caso ideal -> 50% (Positivo - 1) - 50% (Negativo - 0)
data['SP'].unique()    #Clases existentes
print("Próximo pago en mora")
data['SP'].value_counts()

Próximo pago en mora


SP
No    23364
Si     6636
Name: count, dtype: int64

### Datos NaN o faltantes '?'

In [20]:
datos_faltantes = data.isnull()

for i in datos_faltantes.columns.values.tolist():
    print(datos_faltantes[i].value_counts())

ID
False    30000
Name: count, dtype: int64
CU
False    30000
Name: count, dtype: int64
G
False    29997
True         3
Name: count, dtype: int64
ED
False    30000
Name: count, dtype: int64
EC
False    29999
True         1
Name: count, dtype: int64
E
False    30000
Name: count, dtype: int64
M1
False    30000
Name: count, dtype: int64
M2
False    30000
Name: count, dtype: int64
M3
False    30000
Name: count, dtype: int64
M4
False    30000
Name: count, dtype: int64
M5
False    30000
Name: count, dtype: int64
M6
False    30000
Name: count, dtype: int64
D1
False    30000
Name: count, dtype: int64
D2
False    30000
Name: count, dtype: int64
D3
False    30000
Name: count, dtype: int64
D4
False    30000
Name: count, dtype: int64
D5
False    30000
Name: count, dtype: int64
D6
False    30000
Name: count, dtype: int64
P1
False    30000
Name: count, dtype: int64
P2
False    30000
Name: count, dtype: int64
P3
False    30000
Name: count, dtype: int64
P4
False    30000
Name: count, dtype: int64
P5
F

### Datos Anómalos o Cero

In [48]:
# Columna 'ED'
data_sec1 = data['ED']
data_anom1 = pd.DataFrame(data_sec1.isin([0,5,6]))
print(data_anom1.value_counts(),"\n")

# Columna 'EC'
data_sec2 = data['EC']
data_anom2 = pd.DataFrame(data_sec2.isin(['0']))
print(data_anom2.value_counts(),"\n")

# Columna 'M1-M6'
col_anom = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
for i in col_anom:
    data_sec3 = data[i]
    data_anom3 = pd.DataFrame(data_sec3.isin([-2, 0]))
    print(data_anom3.value_counts(),"\n")
    

# Columna 'D1-D6'
col_anom1 = ['D1', 'D2', 'D3', 'D4', 'D5', 'D6']
for i in col_anom1:
    data_sec4 = data[i]
    data_anom4 = pd.DataFrame(data_sec4 <0)
    print(data_anom4.value_counts(),"\n")
    
# Columna 'P1-P6'
col_anom2 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6']
for i in col_anom2:
    data_sec5 = data[i]
    data_anom5 = pd.DataFrame(data_sec5 <0)
    print(data_anom5.value_counts(),"\n")

ED   
False    29655
True       345
Name: count, dtype: int64 

EC   
False    29947
True        53
Name: count, dtype: int64 

M1   
True     17496
False    12504
Name: count, dtype: int64 

M2   
True     19512
False    10488
Name: count, dtype: int64 

M3   
True     19849
False    10151
Name: count, dtype: int64 

M4   
True     20803
False     9197
Name: count, dtype: int64 

M5   
True     21493
False     8507
Name: count, dtype: int64 

M6   
True     21181
False     8819
Name: count, dtype: int64 

D1   
False    29410
True       590
Name: count, dtype: int64 

D2   
False    29331
True       669
Name: count, dtype: int64 

D3   
False    29345
True       655
Name: count, dtype: int64 

D4   
False    29325
True       675
Name: count, dtype: int64 

D5   
False    29345
True       655
Name: count, dtype: int64 

D6   
False    29312
True       688
Name: count, dtype: int64 

P1   
False    30000
Name: count, dtype: int64 

P2   
False    30000
Name: count, dtype: int64 

P3   


## 3. Imputar datos   


In [100]:
# Crear una copia del dataset original
data_mod = data.copy()

# Categórico 'G'
mas_freq = data_mod['G'].value_counts().idxmax()
print(f"Variable más frecuente de G: {mas_freq}")
data_mod.replace({'G': np.nan}, mas_freq, inplace=True)     #Imputar la variable

# Categórico 'EC'
x = 'EC'
mas_freq = data_mod[x].value_counts().idxmax()
print(f"Variable más frecuente de {x}: {mas_freq}")
data_mod[x].replace([np.nan, '0' ], mas_freq, inplace=True)     #Imputar la variable

# Numérico 'ED'
x = 'ED'
print(f"La variable más adecuada de {x} es 4")
data_mod[x].replace([0, 5, 6], 4, inplace=True)

# Numérico 'M1-M6'
for i in col_anom:
    x = i
    #mas_freq = data_mod[x].value_counts().idxmax()
    # M_mediana = round(np.mean(data_mod[x]))           Tanto la media como mediana y la moda dan valores de 0 por lo que no se pueden aplicar en este caso
    print(f"En {x} se remplaza 0 y -2 por 1 y -1 respectivamente \n")
    data_mod[x].replace([0, -2], [1, -1] , inplace=True)
    
# Numérico 'D1-D6'
for i in col_anom1:
    x = i
    neg = data_mod[x]<0
    print(f"En {x} se remplaza los datos negativos por positivos \n")
    data_mod[x]=data_mod[x].abs()


Variable más frecuente de G: F
Variable más frecuente de EC: Soltero
La variable más adecuada de ED es 4
En M1 se remplaza 0 y -2 por 1 y -1 respectivamente 

En M2 se remplaza 0 y -2 por 1 y -1 respectivamente 

En M3 se remplaza 0 y -2 por 1 y -1 respectivamente 

En M4 se remplaza 0 y -2 por 1 y -1 respectivamente 

En M5 se remplaza 0 y -2 por 1 y -1 respectivamente 

En M6 se remplaza 0 y -2 por 1 y -1 respectivamente 

En D1 se remplaza los datos negativos por positivos 

En D2 se remplaza los datos negativos por positivos 

En D3 se remplaza los datos negativos por positivos 

En D4 se remplaza los datos negativos por positivos 

En D5 se remplaza los datos negativos por positivos 

En D6 se remplaza los datos negativos por positivos 



C:\Users\marti\AppData\Local\Temp\ipykernel_9076\2946275098.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_mod[x].replace([np.nan, '0' ], mas_freq, inplace=True)     #Imputar la variable
C:\Users\marti\AppData\Local\Temp\ipykernel_9076\2946275098.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting value

In [101]:
# Verificar rango de dataset imputado
#for x in data_mod.columns:
#    print(x)
#    print(data_mod[x].unique(), "\n")

#Verificar tipo de datos
data_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      30000 non-null  int64  
 1   CU      30000 non-null  int64  
 2   G       30000 non-null  object 
 3   ED      30000 non-null  int64  
 4   EC      30000 non-null  object 
 5   E       30000 non-null  int64  
 6   M1      30000 non-null  int64  
 7   M2      30000 non-null  int64  
 8   M3      30000 non-null  int64  
 9   M4      30000 non-null  int64  
 10  M5      30000 non-null  int64  
 11  M6      30000 non-null  int64  
 12  D1      30000 non-null  float64
 13  D2      30000 non-null  float64
 14  D3      30000 non-null  float64
 15  D4      30000 non-null  float64
 16  D5      30000 non-null  float64
 17  D6      30000 non-null  float64
 18  P1      30000 non-null  float64
 19  P2      30000 non-null  float64
 20  P3      30000 non-null  float64
 21  P4      30000 non-null  float64
 22

#### Cambiar los tipos de datos 



In [102]:
#Transformar datos tipo "object" a "numeric" de columnas 'G' 'EC' 'SP'
# Columna 'G'
data_num1 = pd.get_dummies(data_mod['G'])
data_mod = pd.concat([data_mod, data_num1], axis = 1)
data_mod = data_mod.drop(columns=['G'])

# Columna 'EC'
data_num2 = pd.get_dummies(data_mod['EC'])
data_mod = pd.concat([data_mod, data_num2], axis = 1)
data_mod = data_mod.drop(columns=['EC'])

# Columna 'SP'
map = {'No':0, 'Si':1}
data_mod['SP'] = data_mod['SP'].map(map)

In [99]:
data_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 29 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       30000 non-null  int64  
 1   CU       30000 non-null  int64  
 2   ED       30000 non-null  int64  
 3   E        30000 non-null  int64  
 4   M1       30000 non-null  int64  
 5   M2       30000 non-null  int64  
 6   M3       30000 non-null  int64  
 7   M4       30000 non-null  int64  
 8   M5       30000 non-null  int64  
 9   M6       30000 non-null  int64  
 10  D1       30000 non-null  float64
 11  D2       30000 non-null  float64
 12  D3       30000 non-null  float64
 13  D4       30000 non-null  float64
 14  D5       30000 non-null  float64
 15  D6       30000 non-null  float64
 16  P1       30000 non-null  float64
 17  P2       30000 non-null  float64
 18  P3       30000 non-null  float64
 19  P4       30000 non-null  float64
 20  P5       30000 non-null  float64
 21  P6       300

## 4. Generación de modelos

### Conjunto de entrenamiento y validación

In [103]:
# Utilizando Dataset con datos imputados: mediana
x = data_mod.drop('SP', axis = 1)  #Matriz de entrada
y = data_mod['SP'] #Target - Vector de salida

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 40)   #Generar conjunto de entrenamiento y validación

### Modelo Bagging

In [104]:
# Crear el modelo
modelo_rf = RandomForestClassifier(random_state = 42)

# Entrenar el modelo
modelo_rf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

### Modelo Boosting

In [105]:
# Crear el modelo
modelo_xgb = XGBClassifier(random_state = 42)

# Entrenar el modelo
modelo_xgb.fit(x_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

## 5. Métrica

### Métrica modelo Bagging

In [106]:
# Validar el modelo
y_pred1 = modelo_rf.predict(x_test)

# Matriz de confusión
con_mat1 = confusion_matrix(y_test, y_pred1)
print(f"Matriz de confusión: \n {con_mat1}")

# Exactitud
exactitud1 = accuracy_score(y_test, y_pred1)

# Precisión
precision1 = precision_score(y_test, y_pred1)

# Sensibilidad
recall1 = recall_score(y_test, y_pred1)

# Especifidad
vn1, fp1, fn1, vp1 = con_mat1.ravel()
especifidad1 = vn1/(vn1+fp1)

# f1-score
f1 = f1_score(y_test, y_pred1)


print(f"Exactitud: {exactitud1:.2f}\nPrecisión: {precision1:.2f}\nSensibilidad: {recall1:.2f}\nEspecifidad: {especifidad1:.2f}\nf1-score: {f1:.2f}")

Matriz de confusión: 
 [[6646  373]
 [1225  756]]
Exactitud: 0.82
Precisión: 0.67
Sensibilidad: 0.38
Especifidad: 0.95
f1-score: 0.49


### Métrica modelo Boosting

In [107]:
# Validar el modelo
y_pred2 = modelo_xgb.predict(x_test)

# Matriz de confusión
con_mat2 = confusion_matrix(y_test, y_pred2)
print(f"Matriz de confusión: \n {con_mat2}")

# Exactitud
exactitud2 = accuracy_score(y_test, y_pred2)

# Precisión
precision2 = precision_score(y_test, y_pred2)

# Sensibilidad
recall2 = recall_score(y_test, y_pred2)

# Especifidad
vn2, fp2, fn2, vp2 = con_mat2.ravel()
especifidad2 = vn2/(vn2+fp2)

# f1-score
f1_2 = f1_score(y_test, y_pred2)


print(f"Exactitud: {exactitud2:.2f}\nPrecisión: {precision2:.2f}\nSensibilidad: {recall2:.2f}\nEspecifidad: {especifidad2:.2f}\nf1-score: {f1_2:.2f}")

Matriz de confusión: 
 [[6615  404]
 [1218  763]]
Exactitud: 0.82
Precisión: 0.65
Sensibilidad: 0.39
Especifidad: 0.94
f1-score: 0.48


## 6. Ajuste de hiperparámetros y mejora de métricas

### Modelo Bagging
#### Selección de mejores hiperparámetros con GridSearch

In [112]:
# Diccionario de parámetros a probar
parametros = {"criterion" : ("gini", "entropy"),
              "n_estimators" : (10, 30, 40, 50, 70, 90)}

rejilla = GridSearchCV(modelo_rf,
                       parametros,
                       scoring = "accuracy")  #Al no tener los datos balanceados la mejor métrica a optimizar es "f1" debido a que indica la proporción entre precisión y sensibilidad.

rejilla.fit(x_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'criterion': ('gini', 'entropy'),
                         'n_estimators': (10, 30, 40, 50, 70, 90)},
             scoring='accuracy')

#### Mejor puntuación e hiperparámetros

In [113]:
# Mejor puntuación de la métrica seleccionada
print(rejilla.best_score_)

# Mejores hiperparámetros
print(rejilla.best_params_)

0.815047619047619
{'criterion': 'entropy', 'n_estimators': 70}


#### Entrenamiento de mejor modelo

In [114]:
y_pred1_1 = rejilla.predict(x_test)

# Métricas
con_mat1 = confusion_matrix(y_test, y_pred1_1)
print(f"Matriz de confusión: \n {con_mat1}")

exactitud1 = accuracy_score(y_test, y_pred1_1)

precision1 = precision_score(y_test, y_pred1_1)

recall1 = recall_score(y_test, y_pred1_1)

vn1, fp1, fn1, vp1 = con_mat1.ravel()
especifidad1 = vn1/(vn1+fp1)

f1 = f1_score(y_test, y_pred1_1)

print(f"Exactitud: {exactitud1:.2f}\nPrecisión: {precision1:.2f}\nSensibilidad: {recall1:.2f}\nEspecifidad: {especifidad1:.2f}\nf1-score: {f1:.2f}")

Matriz de confusión: 
 [[6654  365]
 [1219  762]]
Exactitud: 0.82
Precisión: 0.68
Sensibilidad: 0.38
Especifidad: 0.95
f1-score: 0.49


### Modelo Boosting
#### Selección de mejores hiperparámetros con GridSearch

In [115]:
# Diccionario de parámetros a probar
parametros2 = {"booster" : ("dart", "gbtree"),
               "tree_method" : ("exact", "approx", "hist"),
               "scale_pos_weight" : (1.70, 1.86, 1.95),
               "alpha" : ( 5, 8, 9)}

rejilla2 = GridSearchCV(modelo_xgb,
                       parametros2,
                       scoring = "accuracy")

rejilla2.fit(x_train, y_train)

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_b...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             param_grid={'alpha': (5, 8, 9), 'booster': ('dart', 'gbtree'),
                         'scale_pos_weight': (1.7, 1.86, 1.95),
                         'tree_method': ('exact', 'approx', 'hist')},
             scoring='accuracy')

#### Mejor puntuación e hiperparámetros

In [116]:
# Mejor puntuación de la métrica seleccionada
print(rejilla2.best_score_)

# Mejores hiperparámetros
print(rejilla2.best_params_)

0.8032380952380953
{'alpha': 9, 'booster': 'dart', 'scale_pos_weight': 1.7, 'tree_method': 'exact'}


#### Entrenamiento del mejor modelo

In [117]:

y_pred2_2 = rejilla2.predict(x_test)

# Métricas
con_mat2 = confusion_matrix(y_test, y_pred2_2)
print(f"Matriz de confusión: \n {con_mat2}")

exactitud2 = accuracy_score(y_test, y_pred2_2)

precision2 = precision_score(y_test, y_pred2_2)

recall2 = recall_score(y_test, y_pred2_2)

vn2, fp2, fn2, vp2 = con_mat2.ravel()
especifidad2 = vn2/(vn2+fp2)

f1_2 = f1_score(y_test, y_pred2_2)

print(f"Exactitud: {exactitud2:.2f}\nPrecisión: {precision2:.2f}\nSensibilidad: {recall2:.2f}\nEspecifidad: {especifidad2:.2f}\nf1-score: {f1_2:.2f}")

Matriz de confusión: 
 [[6333  686]
 [1017  964]]
Exactitud: 0.81
Precisión: 0.58
Sensibilidad: 0.49
Especifidad: 0.90
f1-score: 0.53


## Guardar mejor modelo

In [118]:
import pickle

#Guardar el modelo
filename = 'Mejor_modelo(DT).sav'
pickle.dump(rejilla, open(filename, 'wb'))
 
#Cargar el modelo
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(x_test, y_test)
#print(result)

## Conclusiones

El mejor modelo es el de RandomForest el cual posee una mayor exactitud, precisión y especifidad con respecto al modelo XGB, sin embargo las métricas obtenidad indican que el modelo predice de mejor manera los casos negativos de diabetes con gran diferencia con respecto a la predicción los casos positivos. 

Por otro lado el modelo de xgb a pesar de tener métricas menores, es más balanceado pues la capacidad de predicción de los casos negativos de diabetes es similar a la capacidad de predicción de los casos positivos.